# Expert Tutorial

ToDo: 
* Create Institute, Site, Collection Protocol,
* create Specimen and dependencies with MasterTemplate,
* update some Data
* delete the Data


In [1]:
%%capture
# Install a pip package in the current Jupyter kernel

import sys
!{sys.executable} -m pip install OpenSpecimenAPIconnector
!{sys.executable} -m pip install faker
!{sys.executable} -m pip install names

In [2]:
%%capture
#Creating ID-dict for deleting the Tutorial Data
ID = {}

In [3]:
%%capture
#import the packages which are needed
import json
import pandas as pd
import ipywidgets as widgets
import faker
import os
from IPython.display import display

### Importing the PIP-package as:

In [4]:
import OpenSpecimenAPIconnector as OSconn
import OpenSpecimenAPIconnector.os_core as os_core
import OpenSpecimenAPIconnector.os_util as os_util

In [5]:
# Setting the URL of the OpenSpecimen distribution
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
input_=input()
if input_ == '':
    base_url = 'http://localhost:9000'
else:
    base_url = input_
base_url +='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter

The whole URL for API Calls is: http://localhost:9000/openspecimen/rest/ng


In [6]:
#Setting the login data, in particular (loginname, passwort)
loginname='admin';
password='Login@123'
print('Loginname(admin):')
input_=input()
if input_ != '':
    loginname=input_
print('Password(Login@123):')
input_=input()
if input_ != '':
    password=input_
auth = (loginname, password)
print('Your (loginname,password) is (' + str(loginname) + ',' +str(password) + ')')

Loginname(admin):

Password(Login@123):

Your (loginname,password) is (admin,Login@123)


### Configuring the package with the Logindata
If the logindata is wrong the next cell will throw an error, else everything is fine.

In [7]:
OSconn.config_manager.set_login(url = base_url, auth = auth)

## Creating Institute, Site and Collection Protocol
With those, the Master-Template can be used for creating Participants, Visits, and Specimens. This will be done with the CSV Export using OpenSpecimen REST API. 

In [8]:
# initialize the classes which will be used
institutes_util = os_util.institutes_util()
site_util = os_util.site_util()
collection_protocol_util = os_util.collection_protocol_util()
bulk_operations = os_util.bulk_operations()
csv_bulk = os_core.csv_bulk()
cp = os_core.collection_protocol()
site = os_core.sites()
institutes = os_core.institutes()
cpevent_util = os_util.cpevent_util()
cpevent = os_core.collection_protocol_event()
specimen_util = os_util.specimen_util()
cpr_util = os_util.cpr_util()
cpr = os_core.collection_protocol_registration()
visit =os_core.visit()

In [9]:
# Generate the Institute, Site and Collection Protocol
response = institutes_util.create_institute(institutename = 'Demo Institute')
ID['institute'] = response['id']
response = site_util.create_sites(name = 'Demo Site', institutename = ' Demo Institute', type_ = 'not specified')
ID['site']=response['id']
response = collection_protocol_util.create_cp(short_title = 'Demo CP', title = 'Demo Collection Protocol',
                                              pi_mail = 'admin', sites = ['Demo Site'])
ID['cp'] = response['id']
response = cpevent_util.create_event(label = 'Demo Event', point = 0, cp = 'Demo Collection Protocol', 
                                     site = 'Demo Site', diagnosis = 'Not Specified', status = 'Not Specified',
                                     activity = 'Active', unit = 'DAYS', code ='DemoEventCode')
ID['event'] = response['id']

In [10]:
df = csv_bulk.get_template(schemaname = 'masterSpecimen')
print(df[0])
df=df[0]

Empty DataFrame
Columns: [CP Short Title, PPID, Registration Date, Registration Site, External Subject ID, First Name, Last Name, Middle Name, Date Of Birth, Death Date, Gender, Race#1, Race#2, Vital Status, Ethnicity#1, Ethnicity#2, SSN, eMPI, Event Label, Visit Name, Visit Date, Collection Site, Visit Status, Clinical Diagnosis#1, Clinical Diagnosis#2, Clinical Status, Path. Number, Visit Comments, Specimen Requirement Code, Specimen Label, Barcode, Class, Type, Lineage, Parent Specimen Label, Anatomic Site, Laterality, Pathological Status, Quantity, Concentration, Freeze/Thaw Cycles, Created On, Comments, Collection Status, Container, Row, Column, Position, Collection Date, Collection Procedure, Collection Container, Collector, Received Date, Received Quality, Receiver, PMI#1#Site Name, PMI#1#MRN, PMI#2#Site Name, PMI#2#MRN, Created By#Email Address, External IDs#1#Name, External IDs#1#Value, External IDs#2#Name, External IDs#2#Value, Specimen Additional Fields#Media Volume, Specime

In [11]:
#write Participant+Specimen dict
pict ={}
pict['CP Short Title'] = 'Demo CP'
pict['PPID'] = 'Demo 001'
pict['Registration Date'] = '03-03-2021'
pict['Date Of Birth'] = '01-01-1901'
pict['Event Label'] = 'Demo Event'
pict['Visit Date'] = '03-03-2021'
pict['Collection Site'] = 'Demo Site'
pict['Visit Status'] = 'Complete'
pict['Clinical Diagnosis#1'] = 'Not Specified'
pict['Clinical Status'] = 'Not Specified'
pict['Specimen Label'] = 'Demo001.WB'
pict['Class'] = 'Fluid'
pict['Type'] = 'Whole Blood'
pict['Lineage'] = 'New'
pict['Anatomic Site'] = 'Blood'
pict['Laterality'] = 'Not Specified'
pict['Pathological Status'] = 'Malignant'
pict['Quantity'] = 5
pict['Created On'] = '03-03-2021'
pict['Collection Date'] = '03-03-2021'
pict['Collector'] = 'admin@localhost'
pict['Received Date'] = '03-03-2021'
pict['Received Quality'] = 'Acceptable'
pict['Receiver'] ='admin@localhost'
pict['PMI#1#Site Name'] = 'Demo Site'
pict['PMI#1#MRN'] = '1111'

In [12]:
#write derivative dict
derict = {}
derict['CP Short Title'] = 'Demo CP'
derict['Specimen Label'] = 'Demo001.Pl'
derict['Class'] = 'Fluid'
derict['Type'] = 'Plasma'
derict['Lineage'] = 'Derived'
derict['Quantity'] = 5
derict['Parent Specimen Label'] = 'Demo001.WB'
derict['Created On'] = '03-03-2021'
#write aliquot
spict=[{},{},{},{}]
j=1
for spi in spict:
    spi['CP Short Title'] = 'Demo CP'
    #spi['Specimen Label'] = 'Demo001.Pl'+ str(j)
    spi['Class'] = 'Fluid'
    spi['Type'] = 'Plasma'
    spi['Quantity'] = 1
    spi['Lineage'] = 'Aliquot'
    spi['Parent Specimen Label'] = 'Demo001.Pl'
    spi['Created On'] = '03-03-2021'

In [13]:
df = df.append(pict, ignore_index = True)
df = df.append(derict, ignore_index = True)
df = df.append(spict, ignore_index = True)
df.to_csv('upload.csv', index = False)
file_ = open('upload.csv', 'r')
response = bulk_operations.bulk_import(file = file_, filename = 'upload.csv',schemaname ='masterSpecimen')
print(response)
print('Press enter to continue.')
input()

  CP Short Title      PPID Registration Date Registration Site  \
0        Demo CP  Demo 001        03-03-2021                 0   
1        Demo CP         0                 0                 0   
2        Demo CP         0                 0                 0   
3        Demo CP         0                 0                 0   
4        Demo CP         0                 0                 0   
5        Demo CP         0                 0                 0   

  External Subject ID First Name Last Name Middle Name Date Of Birth  \
0                   0          0         0           0    01-01-1901   
1                   0          0         0           0             0   
2                   0          0         0           0             0   
3                   0          0         0           0             0   
4                   0          0         0           0             0   
5                   0          0         0           0             0   

  Death Date  ... PMI#2#MRN Crea

In [14]:
# Get all specimens which were imported before
response = specimen_util.search_specimens(label = 'Demo001')
specid = []
for resp in response:
    specid.append(resp['id'])
response = cpr_util.get_registrations(ppid = 'Demo 001')
ID['cpr'] = response[0]['cprId']

In [15]:
#Update a Specimen-label
response = specimen_util.search_specimens(label = 'Demo001.WB')
id_ = response[0]['id']
derict['Type'] = 'Bile' 
response = specimen_util.update_specimen(specimenid=id_, specimentype = derict['Type'], initqty = 5, avaqty = 1,
                                         userid = 2)
print(response)
print('Press enter to continue.')

{'opComments': None, 'id': 7, 'cpId': 2, 'cprId': 2, 'ppid': 'Demo 001', 'eventId': None, 'eventCode': None, 'eventLabel': None, 'visitId': 2, 'visitName': 'Demo 001_Demo Event_2', 'visitStatus': 'Complete', 'sprNo': None, 'visitDate': 1614729600000, 'cpShortTitle': 'Demo CP', 'reqId': None, 'sortOrder': None, 'label': 'Demo001.WB', 'barcode': None, 'type': 'Bile', 'specimenClass': 'Fluid', 'lineage': 'New', 'anatomicSite': 'Blood', 'laterality': 'Not Specified', 'status': 'Collected', 'reqLabel': None, 'pathology': 'Malignant', 'initialQty': 5, 'availableQty': 1, 'concentration': None, 'parentId': None, 'parentLabel': None, 'storageLocation': {'id': -1, 'name': None, 'mode': None, 'positionX': None, 'positionY': None, 'position': None, 'reservationId': None}, 'storageType': None, 'collectionContainer': 'Not Specified', 'collectionDate': 1614729600000, 'storageSite': None, 'activityStatus': 'Active', 'createdOn': 1614729600000, 'createdBy': None, 'code': None, 'distributionStatus': Non

 ## Removing the Tutorial Data

In [17]:
#Ask if the user wants to remove the data
print('Press enter to remove the Tutorial Data. If you like to keep it and delete it manually write anything.')
input_ = input()
if input_ == '':
    specimen_util.delete_specimens(specid)
    visit.delete_visit(ID['event'])
    cpevent.delete_event(ID['event'])
    cpr.delete_participant(ID['cpr'])
    cp.delete_collection_protocol(ID['cp'])   
    site.delete_sites(ID['site'])
    institutes.delete_institute(ID['institute'])
    os.remove('upload.csv')
    
    

Press enter to remove the Tutorial Data. If you like to keep it and delete it manually write anything.

